### Uarray Experiments


- https://github.com/Quansight-Labs/uarray/blob/master/uarray/tests/test_uarray.py
- https://github.com/Quansight-Labs/unumpy/blob/f58b32e3ac469125c900b2bc24735cd103e9be67/unumpy/_multimethods.py#L1108
    - Examples of building front end Numpy API
- https://github.com/Quansight-Labs/unumpy/blob/master/unumpy/numpy_backend.py
    - Examples of backend Numpy API
- Defining domains: https://uarray.org/en/latest/glossary.html#domainglossary

In [4]:
!pip uninstall --yes uarray

Found existing installation: uarray 0.6.0
Uninstalling uarray-0.6.0:
  Successfully uninstalled uarray-0.6.0


In [7]:
#!pip install uarray
!pip install git+git://github.com/Quansight-Labs/uarray.git#egg=uarray

  Cloning git://github.com/Quansight-Labs/uarray.git to /tmp/pip-install-yxbeiim8/uarray
  Running command git clone -q git://github.com/Quansight-Labs/uarray.git /tmp/pip-install-yxbeiim8/uarray
  Created wheel for uarray: filename=uarray-0.6.0+28.ged4307e-cp37-cp37m-linux_x86_64.whl size=247171 sha256=7beee6426142bb2ec5f4a823b7b357f4099e895ac504ce4eaaae668697c47d7b
  Stored in directory: /tmp/pip-ephem-wheel-cache-eob3nmwn/wheels/73/a1/b5/57ee8eb0154b8049769a6b9301a83dff2e2b99a6d17b72e8e8
Successfully built uarray


In [10]:
import dask.array as da

In [2]:
import uarray as ua
from uarray import Dispatchable, wrap_single_convertor

-----

### Example 1

A very simple frontend + backend implementation.

In [6]:
# This is the method that would be documented (i.e. the frontend)
# * Note that neither this nor argreplacer are called if __ua_convert__ is not implemented in backend,
# instead the multimethod is handed to the __ua_function__ method to resolve everything in one place

def argextractor(arg1):
    """ Target API function (docs copied from here to multimethod)"""
    print('in extractor: ', arg1)
    return [ua.Dispatchable(arg1, str)]

def argreplacer(args, kwargs, dispatchables):
    print('in replacer: ', args, kwargs, dispatchables)
    out_args = dispatchables[:1]
    return out_args, kwargs

mm = ua.generate_multimethod(
    argextractor, 
    argreplacer, 
    domain='tests',
    default=lambda arg1: 'default value'
)

class Backend:
    __ua_domain__ = "tests"
    
    def __ua_convert__(self, dispatchables, coerce=False):
        # See: https://github.com/Quansight-Labs/uarray/blob/059dadec639d6317ac40aa065de3b5c2675b2e16/uarray/__init__.py#L67
        print('in convert: ', dispatchables, coerce)
        for i, d in enumerate(dispatchables):
            print('\t' + str(i) + ': ', d.type, d.coercible, d.value)
            yield d.value
            
    def __ua_function__(self, f, args, kwargs):
        print('in func: ', f, args, kwargs) 
        return str(args)
    
be = Backend()

with ua.set_backend(be):
    print(mm('input value'))

in extractor:  input value
in convert:  [<Dispatchable: type=<class 'str'>, value='input value'>] False
	0:  <class 'str'> True input value
in replacer:  ('input value',) {} ('input value',)
in func:  <uarray multimethod 'argextractor'> ('input value',) {}
('input value',)


In [4]:
# Check default with no backend set
print(mm('input value'))

default value


--------

### Example 2

Sub-domains

See: https://github.com/Quansight-Labs/uarray/blob/master/uarray/tests/test_uarray.py#L313

In [4]:
ua.__version__

'0.6.0+28.ged4307e'

In [5]:
import uarray as ua
def test_hierarchical_backends():
    mm = ua.generate_multimethod(
        lambda: (), lambda a, kw, d: (a, kw), "ua_tests.foo.bar"
    )
    subdomains = "ua_tests.foo.bar".split(".")
    depth = len(subdomains)

    mms = [
        ua.generate_multimethod(
            lambda: (), lambda a, kw, d: (a, kw), ".".join(subdomains[: i + 1])
        )
        for i in range(depth)
    ]

    class DisableBackend:
        def __init__(self, domain):
            self.__ua_domain__ = domain
            self.active = True
            self.ret = object()

        def __ua_function__(self, f, a, kw):
            if self.active:
                return self.ret

            raise ua.BackendNotImplementedError(self.__ua_domain__)

    be = [DisableBackend(".".join(subdomains[: i + 1])) for i in range(depth)]

    ua.set_global_backend(be[1])
    for i in range(1, depth):
        assert mms[i]() is be[1].ret
test_hierarchical_backends()

In [4]:
import uarray as ua

def create_array(a, n=10):
    """Multiply arrays"""
    print('in extractor: ', a, b)
    pass


class Backend1:
    __ua_domain__ = "main"
    
    def __ua_function__(self, f, args, kwargs):
        return 'from b1'
    
class Backend2:
    __ua_domain__ = "main.sub1"
    
    def __ua_function__(self, f, args, kwargs):
        return 'from b2'
    
class Backend3:
    __ua_domain__ = "main.sub1.sub2"
    
    def __ua_function__(self, f, args, kwargs):
        return 'from b3'

b1 = Backend1()
b2 = Backend2()
b3 = Backend3()

mm = ua.generate_multimethod(
    create_array,
    lambda args, kwargs, dispatchables: args,
    domain='main.sub1.sub2'
)


ua.clear_backends('main')
#ua.register_backend(b2)
ua.set_global_backend(b2, coerce=True)
print(mm('x', n=5))

BackendNotImplementedError: No selected backends had an implementation for this function.

-------

In [ ]:
# Inside ulinalg, you would define a method consisting of an argument replacer and argument extractor.

# The argument extractor is the simpler of the two: It "extracts" the array arguments from the method as a tuple.
# The argument replacer "replaces" the array arguments inside args, kwargs with the supplied arrays.

# https://github.com/Quansight-Labs/uarray/blob/master/notebooks/02_basic_dev_tutorial.ipynb
def solve_argreplacer(args, kwargs, arrays):
    out_args = arrays + args[2:]
    return out_args, kwargs

@unp.create_numpy(solve_argreplacer)
@all_of_type(unp.ndarray)
def solve(a, b, sym_pos=False, lower=False, overwrite_a=False, overwrite_b=False, debug=None, check_finite=True, assume_a='gen', transposed=False):
    return (a, b)

In [ ]:
# def __ua_function__(method, args, kwargs):
#     if method in _implementations:
#         return _implementations[method](*args, **kwargs)

#     if not hasattr(sparse, method.__name__):
#         return NotImplemented

#     return getattr(sparse, method.__name__)(*args, **kwargs)


# @wrap_single_convertor
# def __ua_convert__(value, dispatch_type, coerce):
#     if dispatch_type is ufunc:
#         return getattr(np, value.name)

#     if value is None:
#         return None

#     if dispatch_type is ndarray:
#         if not coerce:
#             if not isinstance(value, sparse.SparseArray):
#                 return NotImplemented

#         if isinstance(value, sparse.SparseArray):
#             return value

#         return sparse.as_coo(np.asarray(value))